In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1: Read file

In [ ]:
cars = pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")
cars.info()

In [ ]:
cars.head()

Let's look at the distribution of selling price, which is what we are trying to predict (y-variable or the response variable):

In [ ]:
sns.distplot(cars.Selling_Price)

Since it's right-skewed, a transform will normalize the distribution:

In [ ]:
sns.distplot(np.log(cars.Selling_Price), fit=norm, kde=False)

In [ ]:
sns.distplot(np.log1p(cars.Selling_Price), fit=norm, kde=False)

In [ ]:
sns.distplot(np.sqrt(cars.Selling_Price), fit=norm, kde=False)

The natural log transform looks more normally distributed than the others. The present price, then, also needs to be transformed.

In [ ]:
cars.Selling_Price = np.log(cars.Selling_Price)
cars['Present_Price'] = np.log(cars['Present_Price'])

In [ ]:
cars2 = pd.read_csv("../input/vehicle-dataset-from-cardekho/CAR DETAILS FROM CAR DEKHO.csv")
cars2.info()

In [ ]:
cars2.head()

In [ ]:
sns.distplot(cars2.selling_price)

In [ ]:
sns.distplot(np.log(cars2.selling_price), fit=norm, kde=False)

In [ ]:
cars2.selling_price = np.log(cars2.selling_price)

# Step 2: Exploratory Data Analysis (EDA)

## Categorical features

Looking at each categorical feature first and how the average selling price is affected starting with car name:

### Car Name

In [ ]:
cars['Car_Name'].nunique()

In [ ]:
cars2['name'].nunique()

In [ ]:
cars['Car_Name'].value_counts()[:35]

In [ ]:
cars2['name'].value_counts()[:35]

In [ ]:
fig = plt.figure(figsize=(10, 20))

plt.title('Selling price of cars by car name')

price_order = cars.groupby('Car_Name')['Selling_Price'].mean().sort_values(ascending=False).index.values

sns.boxplot(data=cars, x='Selling_Price', y='Car_Name', 
            order=price_order)

In [ ]:
fig = plt.figure(figsize=(10, 20))

plt.title('Selling price of cars by car name')

price_order = cars2.groupby('name')['selling_price'].mean().sort_values(ascending=False).index.values[:75]

sns.boxplot(data=cars2, x='selling_price', y='name', 
            order=price_order)

In [ ]:
mean_price = cars.groupby('Car_Name')['Selling_Price'].mean().reset_index()
mean_price.sort_values(by='Selling_Price', ascending=False).head(5)

### Fuel Type

In [ ]:
def plot_price(feature, title=''):
    fig = plt.figure(figsize=(8, 5))
    
    plt.title(title)

    price_order = cars.groupby(feature)['Selling_Price'].mean().sort_values(ascending=False).index.values

    sns.boxplot(data=cars, y='Selling_Price', x=feature, 
                order=price_order)
    

def plot_price2(feature, title='', width=8, height=5):
    fig = plt.figure(figsize=(width, height))
    
    plt.title(title)

    price_order = cars2.groupby(feature)['selling_price'].mean().sort_values(ascending=False).index.values

    sns.boxplot(data=cars2, y='selling_price', x=feature, 
                order=price_order)

plot_price('Fuel_Type', 'Selling price of cars by fuel type')

In [ ]:
mean_price = cars.groupby('Fuel_Type')['Selling_Price'].mean().reset_index()
mean_price.sort_values(by='Selling_Price', ascending=False)

In [ ]:
plot_price2('fuel', 'Selling price of cars by fuel type')

In [ ]:
mean_price = cars2.groupby('fuel')['selling_price'].mean().reset_index()
mean_price.sort_values(by='selling_price', ascending=False)

### Seller Type

In [ ]:
plot_price('Seller_Type', 'Selling price of cars by seller type')

In [ ]:
mean_price = cars.groupby('Seller_Type')['Selling_Price'].mean().reset_index()
mean_price.sort_values(by='Selling_Price', ascending=False)

In [ ]:
plot_price2('seller_type', 'Selling price of cars by seller type')

In [ ]:
mean_price = cars2.groupby('seller_type')['selling_price'].mean().reset_index()
mean_price.sort_values(by='selling_price', ascending=False)

## Numerical Features

### Year

'Year' looks like a categorical variable since there are multiple different selling prices for each year. It is not a continuous numerical variable. The selling price of the cars based on year below are ordered by the average selling price:

In [ ]:
plot_price('Year', 'Selling price of cars by year')

For an individual year, the sale price distribution looks like this:

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.distplot(cars[cars['Year']==2015].Selling_Price)

In [ ]:
plot_price2('year', 'Selling price of cars by year', width=15)

### Owner

Cars with 0 previous owners (brand new cars) tend to sell for more than cars with one previous owner. There is only one car in the data set with more than 2 owners, therefore, it is not good to compare with brand new cars and cars with one previous owner. This variable can then be one-hot encoded rather than label encoded in case it is not ordinal (ranked).

In [ ]:
plot_price('Owner', 'Selling price of cars by owner')

In [ ]:
plot_price2('owner', 'Selling price of cars by owner')

### Transmission

Cars with automatic transmission have a higher average selling price than cars with manual transmission.

In [ ]:
plot_price('Transmission', 'Selling price of cars by transmission')

In [ ]:
plot_price2('transmission', 'Selling price of cars by transmission')

### Kilometres driven

In [ ]:
cars2.info()

In [ ]:
sns.lmplot(x='Kms_Driven', y='Selling_Price', data=cars)
ax = plt.gca()
ax.set_title('Selling price of cars by kms driven')

There doesn't seem to be a linear relationship between kms driven and selling price. But, we can look at the distribution and see if the feature can be improved.

In [ ]:
sns.distplot(cars['Kms_Driven'])

In [ ]:
sns.distplot(np.sqrt(cars['Kms_Driven']), fit=norm, kde=False)

In [ ]:
sns.distplot(np.log(cars['Kms_Driven']), fit=norm, kde=False)

In [ ]:
cars['Kms_Driven'] = np.log(cars['Kms_Driven'])

sns.lmplot(x='Kms_Driven', y='Selling_Price', data=cars)
ax = plt.gca()
ax.set_title('Selling price of cars by kms driven')

The natural log or square root transform on kms driven gives a better linear relationship on selling price.

In [ ]:
sns.lmplot(x='km_driven', y='selling_price', data=cars2)
ax = plt.gca()
ax.set_title('Selling price of cars by kms driven')

In [ ]:
sns.distplot(cars2['km_driven'])

In [ ]:
cars2['km_driven'] = np.log(cars2['km_driven'])
sns.distplot(cars2['km_driven'])

In [ ]:
sns.lmplot(x='km_driven', y='selling_price', data=cars2)
ax = plt.gca()
ax.set_title('Selling price of cars by kms driven')

### Present Price



In [ ]:
sns.lmplot(x='Present_Price', y='Selling_Price', data=cars)
ax = plt.gca()
ax.set_title('Selling price of cars by present price')

There's no present price in the cars2 data set.

# Step 3: Cleaning up the data

In this step, choose either label encoding or one-hot encoding for the categorical variables. I decided to one-hot encode all the categorical variables except for 'Car_Name'. The 'Year' will also be one-hot encoded since there are difference prices for each year. There are no missing values.

### One-hot Encoding

Concatenate the one-hot encoded columns, drop the columns that were one-hot encoded. Version 1 of one-hot encoding:

In [ ]:
cars = pd.concat([cars, pd.get_dummies(cars['Year'],prefix='Year')], axis=1)
cars.drop(['Year'],axis=1,inplace=True)

In [ ]:
cars2 = pd.concat([cars2, pd.get_dummies(cars2['year'],prefix='Year')], axis=1)
cars2.drop(['year'],axis=1,inplace=True)

A different way of one-hot encoding:

In [ ]:
new_fuel = pd.get_dummies(cars['Fuel_Type'],prefix='Fuel')
new_seller = pd.get_dummies(cars['Seller_Type'],prefix='Seller')
new_transmission = pd.get_dummies(cars['Transmission'],prefix='Transmission')
new_owner = pd.get_dummies(cars['Owner'],prefix='Owner')

frames = [cars, new_fuel, new_seller, new_transmission, new_owner]
temp = pd.concat(frames, axis=1)
temp.drop(['Fuel_Type','Seller_Type','Transmission','Owner'],axis=1,inplace=True)
cars = temp
cars.head()

In [ ]:
new_fuel = pd.get_dummies(cars2['fuel'],prefix='Fuel')
new_seller = pd.get_dummies(cars2['seller_type'],prefix='Seller')
new_transmission = pd.get_dummies(cars2['transmission'],prefix='Transmission')
# new_owner = pd.get_dummies(cars2['owner'],prefix='Owner')

# frames = [cars2, new_fuel, new_seller, new_transmission, new_owner]
frames = [cars2, new_fuel, new_seller, new_transmission]
temp = pd.concat(frames, axis=1)
temp.drop(['fuel','seller_type','transmission'],axis=1,inplace=True)
cars2 = temp
cars2.head()

### Label Encoding

As stated previously, the 'Car_Name' will be label encoded since machine learning models only use numerical features in training and predicting.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(cars['Car_Name'])
cars['Car_Name'] = le.fit_transform(cars['Car_Name'])
cars.head()

In [ ]:
cars.info()

In [ ]:
le2 = LabelEncoder()
le2.fit(cars2['name'])
cars2['name'] = le.fit_transform(cars2['name'])
le2.fit(cars2['owner'])
cars2['owner'] = le.fit_transform(cars2['owner'])
cars2.head()

In [ ]:
cars2.info()

# Step 4: Training and Predicting

In this step, different models will be used to train on the features and then tested on how accurately they can predict the sale price of the car.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import xgboost as xgb

## Predictions for the first data set

In [ ]:
y = cars.Selling_Price
X = cars.drop(columns='Selling_Price', axis=1)
X.head(10)

In [ ]:
pd.DataFrame(y).head(10)

**Train, test, split**

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)

In [ ]:
def final_predictions(model, name):
    new_model = model.fit(X_train, y_train)
    pred = np.exp(model.predict(X_valid))
    print("============= %s and Shuffle Split =============" %name)
    print("Accuracy: %f" %(r2_score(np.exp(y_valid), pred)))
    print("MSE: %f" %(mean_squared_error(np.exp(y_valid), pred)))
    print("MAE: %f" %(mean_absolute_error(np.exp(y_valid), pred)))
    
    new_model = model.fit(X, y)
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    cvs = cross_val_score(model, X, y, cv=cv)
    print('Shuffle and cross validate: %s \nAverage: %.2f' %(cvs, cvs.mean()))

**Linear Regression and Cross Validation**

Since the selling price was transformed using np.log, np.exp is used to transform it back to its original values and then compared with the predictions:

In [ ]:
lr_model = LinearRegression()
final_predictions(lr_model, 'Linear Regression')

**Random Forest and Cross Validation**

In [ ]:
rf_model = RandomForestRegressor(random_state=0)
final_predictions(rf_model, 'Random Forest Regressor')

**Decision Tree and Cross Validation**

In [ ]:
dt_model = DecisionTreeRegressor(random_state=0)
final_predictions(dt_model, 'Decision Tree Regressor')

**XGBoost Regressor**

In [ ]:
xgb_model = xgb.XGBRegressor()
final_predictions(xgb_model, 'XGBoost Regressor')

## Predictions for the second data set

This data set does not include the 'Present_Price' feature. Therefore, the models are less accurate. With the 'Present_Price' LinearRegression was the best model, without it XGBoost is the best model.

In [ ]:
y = cars2.selling_price
X = cars2.drop(columns='selling_price', axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)
X.head(10)

In [ ]:
pd.DataFrame(y).head(10)

In [ ]:
lr_model = LinearRegression()
final_predictions(lr_model, 'Linear Regression')

In [ ]:
rf_model = RandomForestRegressor(random_state=0)
final_predictions(rf_model, 'Random Forest Regressor')

In [ ]:
dt_model = DecisionTreeRegressor(random_state=0)
final_predictions(dt_model, 'Decision Tree Regressor')

In [ ]:
xgb_model = xgb.XGBRegressor()
final_predictions(xgb_model, 'XGBoost Regressor')